In [1]:
import tensorflow as tf
import numpy as np
import src

In [2]:
problem = src.SortingProblem()

In [3]:
tf.reset_default_graph()
network = src.PointerNetwork('cfg')
network.makeGraph()
seqLen = network.maxTimeSteps
batchSize = network.batchSize

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
trainingParams = {}
trainingParams['batchSize'] = batchSize
trainingParams['length'] = seqLen

losses = []

# train on 100 batches of 100 samples each
saver = tf.train.Saver()
with tf.Session() as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    
    # write out log for graph visualization
    writer = tf.summary.FileWriter('checkpoints/', sess.graph)

    for batch in range(100):
        # make current batch of training data
        inputs = problem.makeBatch(trainingParams)
        trainingInputs, targets = problem.makeTargets(inputs)
        
        # apply one training step
        _, loss = sess.run([network.trainOp, network.loss], 
                           feed_dict={network.train: True,
                                      network.rawInputs: inputs,
                                      network.targets: targets,
                                      network.targetInputs: trainingInputs[:,:-1,:]})
        
        # keep track of the loss values
        losses.append(loss)
        
        if batch % 100 == 0 or batch == 99:
            
            # generate a test batch and output its accuracy and also a sample
            print('Batch: ' + str(batch))
            newInputs = problem.makeBatch(trainingParams)
            output = sess.run(network.decoderOutputs, feed_dict={network.train: False,
                                                                 network.rawInputs: newInputs})
            print(newInputs[0, :, :].T[0])
            print(newInputs[0, :, :].T[0][np.array(output).T[0]])
            print(problem.accuracy(solution=np.array(output).T, batchArray=newInputs))
            
            # save checkpoint
            if batch > 0:
                saver.save(sess, 'checkpoints/SortingPointerNetwork', global_step=network.globalStep)

Batch: 0
[ 0.57155973  0.58062273  0.41836503  0.10392872  0.89034588]
[ 0.10392872  0.41836503  0.57155973  0.58062273  0.89034588]
0.41
Batch: 99
[ 0.35785989  0.02675658  0.35261258  0.52242799  0.06660574]
[ 0.02675658  0.06660574  0.35785989  0.35261258  0.52242799]
0.93


The above cells train the sorting problem according to max time in cfg (default is 5).

Now, look at the accuracy of using these network weights on sorting 15 numbers and see how the network performs.

In [ ]:
tf.reset_default_graph()
network = src.PointerNetwork('cfg')
network.maxTimeSteps = 15
network.makeGraph()

In [ ]:
trainingParams = {}
trainingParams['batchSize'] = network.batchSize
trainingParams['length'] = network.maxTimeSteps
saver = tf.train.Saver()
with tf.Session() as sess:
    # initialize variables
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints/'))

    # generate a test batch and output its accuracy and also a sample
    newInputs = problem.makeBatch(trainingParams)
    output = sess.run(network.decoderOutputs, feed_dict={network.train: False,
                                                         network.rawInputs: newInputs})
    print(newInputs[0, :, :].T[0])
    print(newInputs[0, :, :].T[0][np.array(output).T[0]])
    print(problem.accuracy(solution=np.array(output).T, batchArray=newInputs))

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(losses)
plt.show()